In [ ]:
import pandas as pd
import numpy as np
import pickle as pk
import seaborn as sns
from collections import deque
import os
import gc
import warnings
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import deque

warnings.filterwarnings('ignore')

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
%%time
# data load
f = open('./data/train.bin', 'rb')
tr = pk.load(f)
f.close
f = open('./data/test.bin', 'rb')
te = pk.load(f)
f.close()
sub = pd.read_csv('./data/sample_submission.csv')

tr['event_contents'] = tr['event_contents'].replace(np.nan, 'NA')
te['event_contents'] = te['event_contents'].replace(np.nan, 'NA')

target = tr.groupby('game_id').mean()['winner']


In [ ]:
tr_n = np.unique(tr['game_id'])
te_n = np.unique(te['game_id'])
temp_tr = tr[tr['event'] == 0]
temp_te = te[te['event'] == 0]

test = tr[tr['event'] == 2]

# match = []
# for i in range(len(tr_j)):
#     if tr_j['p1'].values[i] == tr_j['p2'].values[i] == 0:
#          match.append(0)
#     elif tr_j['p1'].values[i] == 0 and tr_j['p2'].values[i] == 1:
#         match.append()

In [ ]:
%%time

hist = [[] for _ in range(len(np.unique(tr['game_id'])))]
for i in tqdm(range(len(np.unique(tr['game_id'])))):
    temp = []
    targ = test[test['game_id'] == i]
    for j in range(len(targ)):
        tar = targ['event_contents'].values[j]
        if 'at' == tar[0:2]:
            idx = targ['game_id'].values[j]
            x = tar.replace(' ', '').split('at')
            x = x[1:][0]
            x = x.split(',')
            x1 = x[0][1:]
            x2 = x[1][0:-1]
            temp.append((targ['player'].values[j], float(x1), float(x2)))
    hist[i] = temp
#     if len(temp) >= 2:
#         hist[i] = silhouette_score(temp, KMeans(2).fit_predict(temp))
#         pl_hist.append(test['player'].values[i])
#         if 'Location' == tar[0:len('Location')]:
#             x = tar.replace(' ', '').split('Location')
#             x = x[1:][0]
#             x = x.split(',')
#             x1 = x[0][2:]
#             x2 = x[1][0:-2]
#             hist.append((x1, x2))
#             pl_hist.append(temp_tr['player'].values[i])

In [ ]:
for i in range(38000):
    temp = tr[tr['game_id'] == i]
    hist = []
    for i in range(len(temp)):
        tar = temp['event_contents'].values[i]
#         if 'at' == tar[0:2]:
#             x = tar.replace(' ', '').split('at')
#             x = x[1:][0]
#             x = x.split(',')
#             x1 = x[0][1:]
#             x2 = x[1][0:-1]
#             hist.append((temp['player'].values[i], x1, x2))
        if 'Location' == tar[0:len('Location')]:
            x = tar.replace(' ', '').split('Location')
            x = x[1:][0]
            x = x.split(',')
            x1 = x[0][2:]
            x2 = x[1][0:-2]
            hist.append((temp['player'].values[i], x1, x2))

    mapp = np.zeros((200, 200))

    for item in hist:
        a, b, c = item
        b = round(float(b))
        c = round(float(c))

        if a == 0:
            mapp[b, c] = 1
        else:
            mapp[b, c] = -1
    print(target.values[i])
    plt.imshow(mapp, cmap='hot', interpolation="nearest")
    plt.show()

In [ ]:
tr